# Phylotrackpy with synchronous generations example (completed)

This notebook contains a simple evolving system with synchronous generations, which is typical for most evolutionary computing systems. 
That is, each generation, a population of candidate solutions are evaluated and selected to reproduce. 
This process repeats for a desired number of generations. 

In this example, we implement the one-max problem where individuals are binary strings, and a solution has a genome of all 1s. 
The example code in this notebook incorporates phylogeny tracking.
Below, we link to some existing examples and the `phylotrackpy` documentation to 
for your reference. 
If you're working on this during the tutorial at ALife 2024, feel free to ask for our help in understanding the code! 
We're happy to walk you through anything!  

## Helpful reference material

- `phylotrackpy` documentation: <https://phylotrackpy.readthedocs.io/en/latest/introduction.html>
- `phylotrackpy` GitHub repository: <https://github.com/emilydolson/phylotrackpy>
- Example code from an ALife 2023 tutorial: <https://github.com/emilydolson/alife-phylogeny-tutorial/blob/main/perfect_tracking_final.ipynb> 
  - Scroll down to the `phylotrackpy` heading!

## Setup

First, install required Python packages for this example (e.g., phylotrackpy).
If you are running this locally, we recommend making a Python virtual environment to keep your local Python installation clean: 

```
python -m venv venv
source venv/bin/activate
```

In [ ]:
!python -m pip install -r requirements.txt

In [1]:
# Imports
from phylotrackpy import systematics
import random
import polars as pl

# Seed random number generator
random.seed(8)

## System implementation

Here, the `Organism` class (below) defines candidate solutions with binary genomes, an evaluated fitness value, and a taxon ID. 
The taxon ID member variable keeps track of the organism's taxon in the phylogeny, which is helpful for `phylotrackpy`'s phylogeny tracker.  

In [7]:
class Organism:
    def __init__(
        self,
        num_genes:int = 10,
        randomize_genome:bool = False
    ):
        # Genomes are vectors of binary values
        self.genome = [bool(random.randint(0, 1)) if randomize_genome else False for _ in range(num_genes)]
        # Evaluate initial fitness
        self.EvalFitness()
        # Organisms keep track of their taxon id (useful for phylogeny tracking)
        self.taxon_id = None

    @classmethod
    def FromGenome(cls, genome:list):
        # Create new organism from a given genome.
        org = cls(
            num_genes = 0,
            randomize_genome = False
        )
        org.SetGenome(genome)
        org.EvalFitness()
        return org

    def GetGenome(self):
        return self.genome

    def SetGenome(self, genome:list):
        self.genome = [gene for gene in genome]
        # Evaluate fitness after updating genome
        self.EvalFitness()

    def GetFitness(self):
        return self.fitness

    def EvalFitness(self):
        self.fitness = sum(self.genome)
        return self.fitness

    def GetTaxonID(self):
        return self.taxon_id

    def SetTaxonID(self, id):
        self.taxon_id = id

    def Mutate(self, per_site_mut_rate:float=0.01):
        num_muts = 0
        for gene_i in range(len(self.genome)):
            if (random.random() <= per_site_mut_rate):
                self.genome[gene_i] = not self.genome[gene_i]
                num_muts += 1
        # Update fitness evaluation after mutations
        self.EvalFitness()
        return num_muts


def TournamentSelect(num_parents:int, population:list, tourn_size:int = 4):
    parent_ids = [None for _ in range(num_parents)]
    candidate_ids = [i for i in range(len(population))]
    for i in range(num_parents):
        tourn_participants = random.sample(candidate_ids, tourn_size)
        parent_ids[i] = max([(population[idx].GetFitness(), idx) for idx in tourn_participants])[1]
    return parent_ids


Here, we implement a basic evolutionary computing loop solving the one-max problem to demonstrate how `phylotrackpy` can be integrated into a traditional evolutionary computing context.

In [9]:
pop_size = 500
generations = 1000
per_site_mut_rate = 0.05
genome_length = 100
tournament_size = 8
print_resolution = 100
summary_output_resolution = 10
phylo_snapshot_resolution = 1000

assert(pop_size > 0)
assert(generations > 0)
assert(per_site_mut_rate >= 0 and per_site_mut_rate <= 1.0)
assert(genome_length > 0)
assert(print_resolution > 0)
assert(summary_output_resolution > 0)

# Create a new Systematics object to track the population's phylogeny
# - Taxa in the phylogeny will be defined by genotypes (i.e., offspring with the same genotype as their parent will belong to the same taxon).
phylo_tracker = systematics.Systematics(
    lambda org: org.GetGenome(),
    store_pos = False
)
# Configure snapshots to output basic information about a taxon
phylo_tracker.add_snapshot_fun(
    lambda tax: str(list(map(int,tax.get_info()))).replace(",", " "),
    "genome",
    "Taxon's genome"
)
phylo_tracker.add_snapshot_fun(
    lambda tax: str(sum(tax.get_info())),
    "fitness",
    "Number of ones in taxon genome"
)
# Initialize phylogeny tracker's update to 0
phylo_tracker.set_update(0)

# Create a list to hold phylo metrics over time
data = []

# Create initial population
population = [Organism(num_genes=genome_length, randomize_genome=False) for _ in range(pop_size)]
# Add initial population to phylogeny tracker
for indiv in population:
    indiv.SetTaxonID(phylo_tracker.add_org(indiv))

for gen in range(0, generations+1):
    if (gen % print_resolution) == 0:
        print(f"---Update {gen}---")
        print(f"  Max fitness={max([indiv.GetFitness() for indiv in population])}")

    # Set phylo tracker's update to current generation
    phylo_tracker.set_update(gen)

    # Individuals are evaluated immediately after reproduction, so no need to evaluate
    # the full population here. In other systems, population evaulation would likely happen here.

    # Select parents to reproduce.
    parent_ids = TournamentSelect(
        num_parents = pop_size,
        population = population,
        tourn_size = tournament_size
    )

    # Create next generation's population by copying parents, mutating offspring,
    # and adding offspring to the phylogeny.
    next_pop = []
    for parent_id in parent_ids:
        next_pop.append(Organism.FromGenome(population[parent_id].GetGenome()))
        next_pop[-1].Mutate(per_site_mut_rate)
        offspring_taxon_id = phylo_tracker.add_org(next_pop[-1], population[parent_id].GetTaxonID())
        next_pop[-1].SetTaxonID(offspring_taxon_id)

    # Snapshot phylogeny
    if (gen % phylo_snapshot_resolution) == 0:
        phylo_tracker.snapshot(f"phylo_snapshot_{gen}.csv")

    # Record data for this update
    if (gen % summary_output_resolution) == 0:
        data.append({
            "generation": gen,
            "max_fit": max([indiv.GetFitness() for indiv in population]),
            "phylo_num_taxa": phylo_tracker.get_num_taxa(),
            "phylo_sum_pairwise_dist": phylo_tracker.get_sum_pairwise_distance(False),
            "phylo_variance_pairwise_dist": phylo_tracker.get_variance_pairwise_distance(False),
            "phylo_phylogenetic_diversity": phylo_tracker.get_phylogenetic_diversity(),
            "phylo_mrca_depth": phylo_tracker.mrca_depth()
        })

    # Remove parents from phylogeny
    for indiv in population:
        phylo_tracker.remove_org(indiv.GetTaxonID())
    population = next_pop

phylo_tracker.snapshot("phylo_snapshot_final.csv")
summary_df = pl.DataFrame(data)
summary_df.write_csv("phylo_summary.csv")

---Update 0---
  Max fitness=0
---Update 100---
  Max fitness=87
---Update 200---
  Max fitness=89
---Update 300---
  Max fitness=88
---Update 400---
  Max fitness=89
---Update 500---
  Max fitness=90
---Update 600---
  Max fitness=88
---Update 700---
  Max fitness=90
---Update 800---
  Max fitness=89
---Update 900---
  Max fitness=88
---Update 1000---
  Max fitness=90
